## Comparisons

This code can execute the following variants of losses:  

1. **Variant 1:**  Purely Physics  
   $L_{\theta} = L_{\text{PDE}}$  
   Use $n_{\text{used}} = 0$  

2. **Variant 2:**  Physics + Data  
   $L_{\theta} = L_{\text{PDE}} + \Sigma_{i=1}^{n_{\text{used}}}\| u_i - \hat{u}_i \|_2^2$  
   Use $n_{\text{used}} \in (0, 300]$   

All the results presented were obtained as follows:
1. By estimating the gradients in the physics-informed loss terms using forward mode automatic differentiation (AD).
2. The output field values at given grid points were computed in one forward pass of the network using the einsum function.

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torchsummary import summary
import torch.distributions as td
import math
from sklearn import metrics
import argparse
import random
import os
import time 
import matplotlib.pyplot as plt
from termcolor import colored
from sklearn.model_selection import train_test_split


import sys
sys.path.append("../..")

from utils.networks import *
from utils.visualizer_misc import *
from utils.forward_autodiff import *
from utils.misc import *

from utils.deeponet_networks_2d import *
from utils.visualizer_2d import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Tag this cell with 'parameters'
# parameters
seed = 0 # Seed number.
n_used = 150 # Number of full training fields used for estimating the data-driven loss term
n_iterations = 80000 # Number of iterations.
use_fourier_features = True
n_fourier = 10 # Number of fourier frequencies considered.
save = True # Save results.

In [ ]:
if save == True:
    resultdir = os.path.join(os.getcwd(),'results','a_Vanilla-NO','seed='+str(seed)+'_n_used='+str(n_used)) 
    if not os.path.exists(resultdir):
        os.makedirs(resultdir)
else:
    resultdir = None

In [ ]:
set_seed(seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Load the data
data = torch.load(os.path.join('..','..','data/2D_Burgers_equation_scalar/Burgers_equation_2D_scalar.pt'))

for key, tensor in data.items():
    print(f"{key}: {tensor.shape}")
    
# Random Initial conditions: Nsamples x 32 x 32, each IC sample is (32 x 32)
# Time evolution of the solution field: Nsamples x 21 x 32 x 32.
# Each field is  21 x 32 x 32, rows correspond to time and other dimensions correspond to the field.
# First row corresponds to solution at t=0 (1st time step)
# and next  row corresponds to solution at t=0.05 (2nd time step) and so on.
# last row correspond to solution at t=1 (21st time step).

In [ ]:
inputs = data['input_samples'].float().to(device)
outputs = data['output_samples'].float().to(device)
t_span = data['t_span'].float().to(device)
x_span = data['x_span'].float().to(device)
y_span = data['y_span'].float().to(device)

L = 1.         # Simulation domain [0, L]^2
T = 1.         # Simulation time [0, T]

nt, nx, ny = len(t_span), len(x_span), len(y_span) # number of discretizations in time, location_x and location_y.

grid = torch.vstack((t_span.repeat_interleave(ny*nx), 
              x_span.flatten().repeat(nt),
              y_span.flatten().repeat(nt))).T
print("Shape of grid:", grid.shape) # (nt*nx*ny, 3)
print("grid:", grid) # time, location_x, location_y

# Split the data into training and testing samples
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=50, random_state=seed)

# Check the shapes of the subsets
print("Shape of inputs_train:", inputs_train.shape)
print("Shape of inputs_test:", inputs_test.shape)
print("Shape of outputs_train:", outputs_train.shape)
print("Shape of outputs_test:", outputs_test.shape)
print('#'*100)

In [ ]:
# Of these full training fields available I am using only n_used fields for estimating the data-driven loss term 
inputs_train_used = inputs_train[:n_used, :, :]
print("Shape of inputs_train_used:", inputs_train_used.shape)
outputs_train_used = outputs_train[:n_used, :, :, :]
print("Shape of outputs_train_used:", outputs_train_used.shape)

In [ ]:
inputs_initial_fields_only = torch.load(os.path.join('..','..','data/2D_Burgers_equation_scalar/Burgers_equation_2D_scalar_initial_fields_only.pt')).to(device)
print(inputs_initial_fields_only.shape)

In [ ]:
def fourier_features(t, x, y, n_fourier):
    pi = math.pi
    result = torch.zeros(t.size(0), 3+(6*n_fourier)).to(device)  # Initialize result tensor

    t = t.squeeze()
    x = x.squeeze()
    y = y.squeeze()

    # Compute the transformation
    result[:, 0] = t
    result[:, 1] = x
    result[:, 2] = y
    for i in range(n_fourier):
        result[:, 6*i + 0 + 3] = torch.cos((i + 1) * pi * t)
        result[:, 6*i + 1 + 3] = torch.sin((i + 1) * pi * t)
        result[:, 6*i + 2 + 3] = torch.cos((i + 1) * pi * x)
        result[:, 6*i + 3 + 3] = torch.sin((i + 1) * pi * x)
        result[:, 6*i + 4 + 3] = torch.cos((i + 1) * pi * y)
        result[:, 6*i + 5 + 3] = torch.sin((i + 1) * pi * y)
    return result

In [ ]:
def fourier_features_spatial(x, y, n_fourier):
    pi = math.pi
    result = torch.zeros(x.size(0), 2+(4*n_fourier)).to(device)  # Initialize result tensor
    
    x = x.squeeze()
    y = y.squeeze()
    
    # Compute the transformation
    result[:, 0] = x
    result[:, 1] = y
    for i in range(n_fourier):
        result[:, 4*i + 0 + 2] = torch.cos((i + 1) * pi * x)
        result[:, 4*i + 1 + 2] = torch.sin((i + 1) * pi * x)
        result[:, 4*i + 2 + 2] = torch.cos((i + 1) * pi * y)
        result[:, 4*i + 3 + 2] = torch.sin((i + 1) * pi * y)
    return result

In [ ]:
"""
input_neurons_branch: Number of input neurons in the branch net.
input_neurons_trunk: Number of input neurons in the trunk net.
p: Number of output neurons in both the branch and trunk net.
"""

p = 128 # Number of output neurons in both the branch and trunk net.

input_neurons_branch = (ny, nx) # Specify input size of image as a tuple (height, width)
n_channels = 1
num_filters = [20, 30, 40]
filter_sizes = [3, 3, 3]
strides = [1]*len(num_filters)
paddings = [0]*len(num_filters)
poolings = [('avg', 2, 2), ('avg', 2, 2), ('avg', 2, 2)]  # Pooling layer specification (type, kernel_size, stride)
end_MLP_layersizes = [150, 150, p]
activation = nn.SiLU() # nn.ReLU() nn.SiLU() #Sin() #nn.LeakyReLU() #nn.Tanh()
branch_net = ConvNet(input_neurons_branch, n_channels, num_filters, filter_sizes, strides, paddings, poolings, end_MLP_layersizes, activation)
branch_net.to(device)
# print(branch_net)
print('BRANCH-NET SUMMARY:')
summary(branch_net, input_size=(n_channels, ny, nx))  # input shape is (channels, height, width)
print('#'*100)

# 3 corresponds to t, x and y
if use_fourier_features == False:
    input_neurons_trunk = 3 # Number of input neurons in the trunk net.
else:
    input_neurons_trunk = 3 + (4*n_fourier) # Number of input neurons in the trunk net.
trunk_net = DenseNet(layersizes=[input_neurons_trunk] + [128]*4 + [p], activation=nn.SiLU()) #Sin() #nn.LeakyReLU() #nn.Tanh()
trunk_net.to(device)
# print(trunk_net)
print('TRUNK-NET SUMMARY:')
summary(trunk_net, input_size=(input_neurons_trunk,))
print('#'*100)

model = Vanilla_NO_model(branch_net, trunk_net)
model.to(device)

In [ ]:
num_learnable_parameters = count_learnable_parameters(branch_net) + count_learnable_parameters(trunk_net)
print("Total number of learnable parameters:", num_learnable_parameters)

In [ ]:
def u_pred(net, inputs, t, x, y):
    if use_fourier_features == False:
        u = net(inputs, torch.hstack([t, x, y])) # (bs, neval)
    elif use_fourier_features == True:
        u = net(inputs,  torch.hstack([t, fourier_features_spatial(x, y, n_fourier)])) # (bs, neval)
    return u

In [ ]:
def loss_pde_residual(net, initial_fields, t, x, y):
    
    u = u_pred(net, initial_fields, t, x, y)
    
    # Using forward automatic differention to estimate derivatives in the physics informed loss
    tangent_t, tangent_x, tangent_y = torch.ones(t.shape).to(device), torch.ones(x.shape).to(device), torch.ones(y.shape).to(device)
    ut  = FWDAD_first_order_derivative(lambda t: u_pred(net, initial_fields, t, x, y), t, tangent_t)  # (bs, neval_c) 
    ux = FWDAD_first_order_derivative(lambda x: u_pred(net, initial_fields, t, x, y), x, tangent_x) # (bs, neval_c)
    uxx = FWDAD_second_order_derivative(lambda x: u_pred(net, initial_fields, t, x, y), x, tangent_x) # (bs, neval_c)
    uy = FWDAD_first_order_derivative(lambda y: u_pred(net, initial_fields, t, x, y), y, tangent_y) # (bs, neval_c)
    uyy = FWDAD_second_order_derivative(lambda y: u_pred(net, initial_fields, t, x, y), y, tangent_y) # (bs, neval_c)
    
    pde_residual = (ut + (u*ux) + (u*uy) - (0.01*uxx) - (0.01*uyy))**2
    
    return torch.mean(pde_residual)

In [ ]:
def loss_pde_bcs(net, initial_fields, t, x, y):
    
    t_left, x_left, y_left = t[0], x[0], y[0]
    t_right, x_right, y_right = t[1], x[1], y[1]
    t_bottom, x_bottom, y_bottom = t[2], x[2], y[2]
    t_top, x_top, y_top = t[3], x[3], y[3]

    u_left = u_pred(net, initial_fields, t_left, x_left, y_left) # u is (bs, neval_b)
    u_right = u_pred(net, initial_fields, t_right, x_right, y_right) # u is (bs, neval_b)
    u_bottom = u_pred(net, initial_fields, t_bottom, x_bottom, y_bottom)  # u is (bs, neval_b)
    u_top = u_pred(net, initial_fields, t_top, x_top, y_top) # u is (bs, neval_b)
    
    # bc1a and bc1b
    pde_bc1a = (u_left - u_right)**2
    tangent_x_left, tangent_x_right = torch.ones(x_left.shape).to(device), torch.ones(x_right.shape).to(device)
    ux_left = FWDAD_first_order_derivative(lambda x_left: u_pred(net, initial_fields, t_left, x_left, y_left), x_left, tangent_x_left) # (bs, neval_b)
    ux_right = FWDAD_first_order_derivative(lambda x_right: u_pred(net, initial_fields, t_right, x_right, y_right), x_right, tangent_x_right) # (bs, neval_b)
    pde_bc1b = (ux_left - ux_right)**2
    
    # bc2a and bc2b
    pde_bc2a = (u_bottom - u_top)**2
    tangent_y_bottom, tangent_y_top = torch.ones(y_bottom.shape).to(device), torch.ones(y_top.shape).to(device)
    uy_bottom = FWDAD_first_order_derivative(lambda y_bottom: u_pred(net, initial_fields, t_bottom, x_bottom, y_bottom), y_bottom, tangent_y_bottom) # (bs, neval_b)
    uy_top = FWDAD_first_order_derivative(lambda y_top: u_pred(net, initial_fields, t_top, x_top, y_top), y_top, tangent_y_top) # (bs, neval_b)
    pde_bc2b = (uy_bottom - uy_top)**2
    
    return torch.mean(pde_bc1a) + torch.mean(pde_bc1b) + torch.mean(pde_bc2a) + torch.mean(pde_bc2b)

In [ ]:
def loss_pde_ic(net, initial_fields, t, x, y):
    
    u_ic = u_pred(net, initial_fields, t, x, y) # u is (bs, neval_i)
    
    bs_ = initial_fields.shape[0]
    ic_values_ = torch.zeros((bs_, x.shape[0], 1)).to(device)
    for j in range(bs_):
        ic_values_[j] = linear_interpolation_2D(x, y, x_span, y_span, initial_fields[j]) # initial condition: u_0(x) values
    ic_values = ic_values_.reshape(-1, x.shape[0]) # (bs, neval_i)
    
    pde_ic = (u_ic - ic_values)**2
    
    return torch.mean(pde_ic)

In [ ]:
def collocation_points(tc_span, xc_span, yc_span, neval_dict):
    tc = tc_span.repeat_interleave(neval_dict['loc']).unsqueeze(-1)
    xc = xc_span.flatten().repeat(neval_dict['t']).unsqueeze(-1)
    yc = yc_span.flatten().repeat(neval_dict['t']).unsqueeze(-1)
    return tc, xc, yc

In [ ]:
start_time = time.time()

bs = 32 # Batch size

neval_t = 21  # Number of randomly chosen time points at which output field is evaluated.
neval_x = neval_y = 64
# neval_loc = neval_x*neval_y  # Number of locations at which output field is evaluated at each time point.

neval_c = {'t': neval_t, 'loc': neval_x*neval_y}  # Number of collocation points within the domain.
neval_b = {'t': neval_t, 'loc': neval_x*1}        # Number of collocation points on each boundary.
neval_i = {'t': 1, 'loc': neval_x*neval_y}        # Number of collocation points at t=0.
        
# Training
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20000, gamma=1.0) # gamma=0.8

iteration_list, loss_list, learningrates_list = [], [], []
datadriven_loss_list, pinn_loss_list = [], []
test_iteration_list, test_loss_list = [], []

for iteration in range(n_iterations):
    
    if n_used == 0:
        datadriven_loss = torch.tensor([0.]).to(device)
        # print('*********')
    else:
        indices_datadriven = torch.randperm(n_used).to(device) # Generate random permutation of indices
        inputs_train_used_batch = inputs_train_used.reshape(-1, 1, ny, nx)[indices_datadriven[0:bs]]
        outputs_train_used_batch = outputs_train_used.reshape(-1, nt*nx*ny)[indices_datadriven[0:bs]]
        # print(f"Shape of inputs_train_used_batch:", inputs_train_used_batch.shape) # (bs, no. of channels, height, width)
        # print(f"Shape of outputs_train_used_batch:", outputs_train_used_batch.shape) # (bs, nt*nx*ny)

        predicted_values = u_pred(model, inputs_train_used_batch, 
                              grid[:, 0].reshape(-1,1), 
                              grid[:, 1].reshape(-1,1), 
                              grid[:, 2].reshape(-1,1))  # (bs, neval) = (bs, nt*nx*ny)
        target_values = outputs_train_used_batch # (bs, nt*nx*ny)
        datadriven_loss = nn.MSELoss()(predicted_values, target_values)
        # print('*********')
    
    num_samples = len(inputs_initial_fields_only)
    indices_pinn = torch.randperm(num_samples).to(device) # Generate random permutation of indices
    inputs_batch = inputs_initial_fields_only.reshape(-1, 1, ny, nx)[indices_pinn[0:bs]]
    # print(f"Shape of inputs_batch:", inputs_batch.shape) # (bs, no. of channels, height, width)

    # points within the domain
    tc_span = td.uniform.Uniform(0., T).sample((neval_c['t'], 1)).to(device)
    xc_span = td.uniform.Uniform(0., L).sample((neval_c['loc'], 1)).to(device)
    yc_span = td.uniform.Uniform(0., L).sample((neval_c['loc'], 1)).to(device)

    tc, xc, yc = collocation_points(tc_span, xc_span, yc_span, neval_c)
    
    # boundary points
    
    # for bc1a and bc1b
    t_bc1_span = td.uniform.Uniform(0., T).sample((neval_b['t'], 1)).to(device)
    x_left_span = torch.full((neval_b['loc'], 1), 0.).to(device)
    x_right_span = torch.full((neval_b['loc'], 1), L).to(device)
    y_bc1_span = td.uniform.Uniform(0, L).sample((neval_b['loc'], 1)).to(device)
    t_left, x_left, y_left = collocation_points(t_bc1_span, x_left_span, y_bc1_span, neval_b)
    t_right, x_right, y_right = collocation_points(t_bc1_span, x_right_span, y_bc1_span, neval_b)
    
    # for bc2a and bc2b
    t_bc2_span = td.uniform.Uniform(0., T).sample((neval_b['t'], 1)).to(device)
    y_bottom_span = torch.full((neval_b['loc'], 1), 0.).to(device)
    y_top_span = torch.full((neval_b['loc'], 1), L).to(device)
    x_bc2_span = td.uniform.Uniform(0, L).sample((neval_b['loc'], 1)).to(device)
    t_bottom, x_bottom, y_bottom = collocation_points(t_bc2_span, x_bc2_span, y_bottom_span, neval_b)
    t_top, x_top, y_top = collocation_points(t_bc2_span, x_bc2_span, y_top_span, neval_b)
    
    tb = [t_left, t_right, t_bottom, t_top]
    xb = [x_left, x_right, x_bottom, x_top]
    yb = [y_left, y_right, y_bottom, y_top]
    
    # initial points
    ti = torch.full((neval_i['loc'], 1), 0.).to(device)
    xi = td.uniform.Uniform(0., L).sample((neval_i['loc'], 1)).to(device)
    yi = td.uniform.Uniform(0., L).sample((neval_i['loc'], 1)).to(device)
    
    pinn_loss = (loss_pde_residual(model, inputs_batch, tc, xc, yc)
                 + loss_pde_bcs(model, inputs_batch, tb, xb, yb) 
                 + loss_pde_ic(model, inputs_batch, ti, xi, yi))
    # print('*********')

    optimizer.zero_grad()
    loss = datadriven_loss + pinn_loss
    loss.backward()
    # torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
    optimizer.step()
    scheduler.step()

    if iteration % 500 == 0:
        # Test loss calculation
        model.eval()  # Set model to evaluation mode
        with torch.no_grad():
            test_predicted_values = u_pred(model, inputs_test.reshape(-1, 1, ny, nx), 
                              grid[:, 0].reshape(-1,1), 
                              grid[:, 1].reshape(-1,1), 
                              grid[:, 2].reshape(-1,1))  # (bs, neval) = (bs, nt*nx*ny)
            test_loss = nn.MSELoss()(test_predicted_values, outputs_test.reshape(-1, nt*nx*ny))
            test_iteration_list.append(iteration)
            test_loss_list.append(test_loss.item())  
        model.train()  # Switch back to training mode
        print('Iteration %s -' % iteration, 'loss = %f,' % loss,
              'data-driven loss = %f,' % datadriven_loss,'pinn loss = %f,' % pinn_loss,
              'learning rate = %f,' % optimizer.state_dict()['param_groups'][0]['lr'], 
              'test loss = %f' % test_loss)

    iteration_list.append(iteration)
    loss_list.append(loss.item())
    datadriven_loss_list.append(datadriven_loss.item())
    pinn_loss_list.append(pinn_loss.item())
    learningrates_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
    
if save == True:
    np.save(os.path.join(resultdir,'iteration_list.npy'), np.asarray(iteration_list))
    np.save(os.path.join(resultdir,'loss_list.npy'), np.asarray(loss_list))
    np.save(os.path.join(resultdir, 'datadriven_loss_list.npy'), np.asarray(datadriven_loss_list))
    np.save(os.path.join(resultdir, 'pinn_loss_list.npy'), np.asarray(pinn_loss_list))
    np.save(os.path.join(resultdir,'learningrates_list.npy'), np.asarray(learningrates_list))
    np.save(os.path.join(resultdir,'test_iteration_list.npy'), np.asarray(test_iteration_list))
    np.save(os.path.join(resultdir, 'test_loss_list.npy'), np.asarray(test_loss_list)) 

plot_loss_terms(resultdir, iteration_list, loss_list, datadriven_loss_list, pinn_loss_list, save)  
    
plot_training_loss(resultdir, iteration_list, loss_list, save) 

plot_testing_loss(resultdir, test_iteration_list, test_loss_list, save)

plot_training_testing_loss(resultdir, iteration_list, loss_list, test_iteration_list, test_loss_list, save)

plot_learningrates(resultdir, iteration_list, learningrates_list, save)  
    
# end timer
end_time = time.time()
training_time = end_time - start_time

runtime_per_iter = training_time/n_iterations # in sec/iter

In [ ]:
if save == True:
    torch.save(model.state_dict(), os.path.join(resultdir,'model_state_dict.pt'))
# model.load_state_dict(torch.load(os.path.join(resultdir, 'model_state_dict.pt'), map_location=device))

In [ ]:
# Predictions
predictions_test = u_pred(model, inputs_test.reshape(-1, 1, ny, nx), 
                          grid[:, 0].reshape(-1,1), 
                          grid[:, 1].reshape(-1,1), 
                          grid[:, 2].reshape(-1,1))  # (bs, neval) = (bs, nt*nx*ny)
# print(predictions_test.shape)

mse_list, r2score_list, relerror_list = [], [], []
    
for i in range(inputs_test.shape[0]):
    
    prediction_i = predictions_test[i].reshape(1, -1) # (1, nt*nx*ny)
    target_i = outputs_test[i].reshape(1, -1) # (1, nt*nx*ny)

    mse_i = F.mse_loss(prediction_i.cpu(), target_i.cpu())
    r2score_i = metrics.r2_score(target_i.flatten().cpu().detach().numpy(), prediction_i.flatten().cpu().detach().numpy()) 
    relerror_i = np.linalg.norm(target_i.flatten().cpu().detach().numpy() - prediction_i.flatten().cpu().detach().numpy()) / np.linalg.norm(target_i.flatten().cpu().detach().numpy())
        
    mse_list.append(mse_i.item())
    r2score_list.append(r2score_i.item())
    relerror_list.append(relerror_i.item())
    
    # Plot the full solution-field for few cases:
    if (i+1) % 5 == 0:
        print(colored('TEST SAMPLE '+str(i+1), 'red'))

        r2score_i = float('%.4f'%r2score_i)
        relerror_i = float('%.4f'%relerror_i)
        print('Rel. L2 Error = '+str(relerror_i)+', R2 score = '+str(r2score_i))
        
        cmap = 'jet'  # Color map
        fontsize = 14  # Font size for labels and titles
        levels = 100
        # Plotting 
        plot_input_field(i, x_span.cpu().detach().numpy(), y_span.cpu().detach().numpy(), inputs_test[i].cpu().detach().numpy(), f"Initial field", cmap, fontsize, levels, resultdir, save)
        plot_solution_field(i, x_span.cpu().detach().numpy(), y_span.cpu().detach().numpy(), target_i.reshape(nt,ny,nx).cpu().detach().numpy(), t_span.cpu().detach().numpy(), f"True Solution", cmap, fontsize, levels, resultdir, save, 'True-Solution')
        plot_solution_field(i, x_span.cpu().detach().numpy(), y_span.cpu().detach().numpy(), prediction_i.reshape(nt,ny,nx).cpu().detach().numpy(), t_span.cpu().detach().numpy(), f"Predicted Solution", cmap, fontsize, levels, resultdir, save, 'Predicted-Solution')
        plot_solution_field(i, x_span.cpu().detach().numpy(), y_span.cpu().detach().numpy(), torch.abs(target_i.reshape(nt,ny,nx) - prediction_i.reshape(nt,ny,nx)).cpu().detach().numpy(), t_span.cpu().detach().numpy(), f"Absolute error", cmap, fontsize, levels, resultdir, save, 'Absolute error')
        print(colored('#'*230, 'green'))

mse = sum(mse_list) / len(mse_list)
print("Mean Squared Error Test:\n", mse)
r2score = sum(r2score_list) / len(r2score_list)
print("R2 score Test:\n", r2score)
relerror = sum(relerror_list) / len(relerror_list)
print("Rel. L2 Error Test:\n", relerror)

In [ ]:
test_dict = {
    "inputs_test": inputs_test.cpu(),
    "outputs_test": outputs_test.cpu(),
    "predictions_test": predictions_test.reshape(-1, nt, ny, nx).cpu()
}
for key, value in test_dict.items():
    print(f"Shape of {key}: {value.shape}")
print(colored('#'*230, 'green'))

if save == True:
    torch.save(test_dict, os.path.join(resultdir,'test_dict.pth'))

In [ ]:
performance_metrics(mse, r2score, relerror, training_time, runtime_per_iter, resultdir, save)